In [145]:
# CMPT459 2020 spring
# milestone2
# data cleaned and feature selection and numericalization for classifier
# group name: Salt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv as csv
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.model_selection import train_test_split

def outlier_fd(data, threshold=0.5):
    low, high = np.percentile(data,1), np.percentile(data, 99)
    dist = high - low
    cut_off = dist * threshold
    lower, upper = low-cut_off, high+cut_off
    return [lower, upper]

def photos_number_counter(x):
    return len(x)

def valid_des(s):
    valid_s = " "
    for i in s.split():
        if i.isalnum():
            valid_s = " ".join([valid_s, i])
            
    return valid_s.strip()





In [146]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')
ptrain = train
ptest = test
ptrain = ptrain.drop(['interest_level'], axis = 1)
pwhole = ptrain.append(ptest)

In [147]:
#Find the outlier of price, latitude and longitude using bound given by outlier_fd
[price_lower, price_higher] = outlier_fd(ptrain['price'])
[latitude_lower, latitude_higher] = outlier_fd(ptrain['latitude'],1)
[longitude_lower, longitude_higher] = outlier_fd(ptrain['longitude'],1)
#deal with outliers, set the column for modified data and plot again
pwhole['price_modify'] = pwhole['price']
pwhole['price_modify'].loc[pwhole['price_modify']<price_lower] = price_lower
pwhole['price_modify'].loc[pwhole['price_modify']>price_higher] = price_higher
#deal with outliers, set the column for modified data and plot again
pwhole['latitude_modify'] = pwhole['latitude']
pwhole['latitude_modify'].loc[pwhole['latitude_modify']<latitude_lower] = latitude_lower
pwhole['latitude_modify'].loc[pwhole['latitude_modify']>latitude_higher] = latitude_higher
#deal with outliers, set the column for modified data and plot again
pwhole['longitude_modify'] = pwhole['longitude']
pwhole['longitude_modify'].loc[pwhole['longitude']<longitude_lower] = longitude_lower
pwhole['longitude_modify'].loc[pwhole['longitude']>longitude_higher] = longitude_higher
#get the number of photos
pwhole['photos_number'] = pwhole['photos'].apply(photos_number_counter)
#set all feature words in tf_idf as features for train for feature_modify
pwhole['features_modify'] = pwhole['features'].apply(lambda x:" ".join(["_".join(i.split(" "))for i in x]))
counterf = CountVectorizer(max_features=100)
train_sparse_f = counterf.fit_transform(pwhole['features_modify'])
feature_list = counterf.get_feature_names()
array1 = train_sparse_f.toarray()
for i, feature_str in enumerate(feature_list):
    list_all = []
    [rows, cols] = array1.shape
    for row in range(rows):
        list_all.append(array1[row][i])
    feature_str_ = feature_str + '(f)'
    feature_v = pd.Series(list_all,pwhole.index, name = feature_str_)
    pwhole[feature_str_] = feature_v
#set all feature words in tf_idf as features for train for description_modify
pwhole['description_modify'] = pwhole['description'].apply(lambda x:valid_des(x))
counterd = CountVectorizer(stop_words=['features'], max_features=100)
train_sparse_d = counterd.fit_transform(pwhole['description_modify'])
description_list = counterd.get_feature_names()
array2 = train_sparse_d.toarray()
for i, description_str in enumerate(description_list):
    list_all = []
    [rows, cols] = array2.shape
    for row in range(rows):
        list_all.append(array2[row][i])
    description_str_ = description_str + '(d)'
    description_v = pd.Series(list_all,pwhole.index, name = description_str_)
    pwhole[description_str_] = description_v
## /*add something to change the created to year month day*/
pwhole['created']=pd.to_datetime(pwhole["created"])
pwhole['year']=pwhole['created'].dt.year
pwhole['month']=pwhole['created'].dt.month
pwhole['day']=pwhole['created'].dt.day



/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [148]:
pwhole = pwhole.drop(['central_a(f)','in_super(f)','on(f)','pre(f)','war(f)'], axis = 1)
pwhole = pwhole.drop(['an(d)','and(d)','are(d)','as(d)','at(d)','be(d)','by(d)','can(d)',
                      'for(d)','from(d)','in(d)','is(d)','it(d)','just(d)','me(d)','more(d)',
                      'natural(d)','new(d)','or(d)','park(d)','that(d)','the(d)','this(d)',
                      'to(d)','will(d)','with(d)','you(d)','your(d)'], axis = 1)
pwhole = pwhole.drop(["building_id", "created", "description", "display_address", "features", 
                     "manager_id", "photos", "street_address", "price", "longitude",
                        "latitude", "features_modify", "description_modify"], axis=1)


In [149]:
X = pwhole[:49352]
X_Test = pwhole[49352:]
X = X.drop(['listing_id'], axis = 1)
X_Test = X_Test.drop(['listing_id'], axis = 1)
Y = pd.get_dummies(train['interest_level'])


In [150]:
X_train

,bathrooms,bedrooms,price_modify,latitude_modify,longitude_modify,photos_number,_photos(f),actual_apt(f),balcony(f),bike_room(f),...,supports(d),text(d),two(d),unit(d),west(d),windows(d),york(d),year,month,day
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1,2400.0,40.7108,-73.9539,12,0,0,0,0,...,0,1,0,0,0,0,0,2016,6,16
1,1.0,2,3800.0,40.7513,-73.9722,6,0,0,0,0,...,0,1,0,0,0,0,0,2016,6,1
3,1.5,3,3000.0,40.7145,-73.9425,5,0,0,0,0,...,0,0,0,0,0,0,0,2016,6,24
4,1.0,0,2795.0,40.7439,-73.9743,4,0,0,0,0,...,0,0,0,0,0,0,0,2016,6,28
5,1.0,3,7200.0,40.7348,-73.9865,5,0,0,0,0,...,0,0,0,0,0,0,0,2016,6,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,3,2800.0,40.8433,-73.9396,5,0,0,0,0,...,1,0,0,0,0,0,1,2016,4,5
49348,1.0,2,2395.0,40.8198,-73.9578,5,0,0,0,0,...,0,0,0,0,0,0,1,2016,4,2
49349,1.0,1,1850.0,40.5765,-73.9554,3,0,0,0,0,...,0,0,0,0,0,0,0,2016,4,26


In [151]:
training_score = 0
validation_score = 0
Best_validation_score = 20
count = 0
total_validation_score = 0
precision = []
recall = []
test_accuracy =[]
test_accuracy_value = 0
test_precision = 0

from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import log_loss
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score
# from sklearn.model_selection import cross_val_score
# scrores
#maybe 5-fold is better than 10-fold
kf = RepeatedKFold(n_splits=5, n_repeats=5, random_state=None) 

for train_index, test_index in kf.split(X):
#     print("Train:", train_index, "Validation:",test_index)
    X['id']=list(range(0,49352))
    X.set_index("id", inplace=True)
    Y['id']=list(range(0,49352))
    Y.set_index("id", inplace=True)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    Y_train, Y_test = Y.loc[train_index,:], Y.loc[test_index,:]
    OVR = OneVsRestClassifier(LogisticRegression(warm_start=True,max_iter=200,tol=0.00001)).fit(X_train,Y_train)
#   classification accuracy metrics
    if test_accuracy_value<OVR.score(X_test,Y_test):
        test_accuracy_value=OVR.score(X_test,Y_test)
        Best_OVR2 = OVR
    test_accuracy.append(OVR.score(X_test,Y_test))
#   log loss on training dataset
    Y_Prediction=OVR.predict_proba(X_train)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_train=pd.DataFrame(Y_train)
    training_score=log_loss(Y_train, Y_Prediction)
    print("training log_loss: %0.3f" % training_score)

#   log loss on test dataset
    Y_Prediction=OVR.predict_proba(X_test)
    Y_Prediction=pd.DataFrame(Y_Prediction)
    Y_test=pd.DataFrame(Y_test)
    validation_score=log_loss(Y_test, Y_Prediction)
    count+=1
    total_validation_score+=validation_score
    print("validation log_loss: %0.3f" % validation_score)

#   logloss metrics
    if validation_score<Best_validation_score:
        Best_validation_score = validation_score
        Best_OVR1 = OVR

    if count==5:
        average = total_validation_score/float(count)
        print("average validation log_loss: %0.3f" % average)
        count=0
        total_validation_score=0
    
    Y_Prediction=OVR.predict(X_test)
#   print confusion matrix
    print(multilabel_confusion_matrix(Y_test, Y_Prediction))
#   confusion matrix matrics
    if test_precision<precision_score(Y_test, Y_Prediction, average='macro'):
        test_precision=precision_score(Y_test, Y_Prediction, average='macro')
        Best_OVR3 = OVR
    precision.append(precision_score(Y_test, Y_Prediction, average='macro'))
    recall.append(recall_score(Y_test, Y_Prediction, average='macro'))
print ("One vs rest best validation accuracy: %0.3f" % Best_validation_score) 
print(precision)
print(recall)
print (test_accuracy)

/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.669
validation log_loss: 0.670
[[[9123   21]
  [ 696   31]]

 [[1091 1912]
  [ 656 6212]]

 [[7419  176]
  [2139  137]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.670
validation log_loss: 0.675
[[[9087   26]
  [ 737   21]]

 [[1113 1866]
  [ 734 6158]]

 [[7444  206]
  [2066  155]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

training log_loss: 0.670
validation log_loss: 0.670
[[[9050   30]
  [ 757   33]]

 [[1120 1868]
  [ 663 6219]]

 [[7463  209]
  [2062  136]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.670
validation log_loss: 0.674
[[[9086   28]
  [ 737   19]]

 [[1152 1834]
  [ 700 6184]]

 [[7465  175]
  [2089  141]]]


/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/lewislin/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


training log_loss: 0.669
validation log_loss: 0.677
average validation log_loss: 0.676
[[[9065   22]
  [ 759   24]]

 [[1076 1870]
  [ 740 6184]]

 [[7472  235]
  [2016  147]]]
One vs rest best validation accuracy: 0.667
[0.5995004944455395, 0.5478730161845459, 0.5313326873366359, 0.5484171326387489, 0.5816973894468057, 0.5360687194897701, 0.5741851158972934, 0.5764504421219564, 0.5524495543538895, 0.5272529636887512, 0.5629844949718843, 0.5191638227594808, 0.5482890238235334, 0.5624658038755506, 0.5509047704799016, 0.5582954584560906, 0.5671394585374276, 0.5730844131063122, 0.557712637132804, 0.5572409525994753, 0.5413884422219847, 0.5710183725196424, 0.5623414723064725, 0.5405741684406064, 0.5581243726780645]
[0.3357729593639767, 0.3303308596358621, 0.33043043369569114, 0.33301051121994973, 0.3328745465630867, 0.32289615496238705, 0.32905085735250733, 0.3352030313807615, 0.332973097077527, 0.33290422921635243, 0.3326541966845106, 0.3280783883848488, 0.3301099258387879, 0.334447528076

In [153]:
Y_Prediction=Best_OVR1.predict_proba(X_Test)  #use the best classifier evaluated by log loss
y=pd.DataFrame(Y_Prediction)
y.reset_index(drop=True, inplace=True)
ids = test['listing_id']
ids.reset_index(drop=True, inplace=True)
out = pd.concat([ids, y],axis=1, ignore_index=True)
out
# out.columns=['listing_id','high','medium','low']
out.columns=['listing_id','high','low','medium']
submission_file = open("submission.csv", "w")
out.to_csv('submission.csv', index = False)
submission_file.close()